<a href="https://colab.research.google.com/github/jabbermcn/Codewars/blob/master/HappyAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Mikalai Mikhailouski (jabbermnc@gmail.com / @jabber_mcn)

In [ ]:
Task 2

In [ ]:
import pygame

pygame.mixer.init()
pygame.mixer.music.load('Pirates of the Caribbean - Main Theme [MIDIfind.com].mid')

pygame.mixer.music.play()

while pygame.mixer.music.get_busy():
    continue


In [ ]:
Task 3

In [ ]:
import gpt_2_simple as gpt2

# Download the GPT-2 model (can be changed to any other GPT-2 model).
gpt2.download_gpt2(model_name='355M')

# Load the GPT-2 model.
model_name = '355M'
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, model_name=model_name)

# Generate text.
text = gpt2.generate(sess, length=100, temperature=0.7, prefix='Product description: ')

# Print the generated text.
print(text)


In [ ]:
Task 4

In [ ]:
# Import necessary libraries
import torch
import torchvision.transforms as transforms
from PIL import Image

from cycleGAN import Generator

# Load pre-trained generator model
generator = Generator()
generator.load_state_dict(torch.load('generator.pth'))
generator.eval()

# Define input image path and output image path
input_path = 'IMG_7945.jpg'
output_path = 'nighttime_image.jpg'

# Load input image and apply transformation
input_image = Image.open(input_path).convert('RGB')
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
input_tensor = transform(input_image).unsqueeze(0)

# Generate output image
with torch.no_grad():
    output_tensor = generator(input_tensor)
output_image = transforms.ToPILImage()(output_tensor.squeeze(0))

# Save output image
output_image.save(output_path)


In [ ]:
Task 5

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

# Load pre-trained VGG19 model
model = models.vgg19(pretrained=True).features
model.eval()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Define image preprocessing function
def preprocess(image):
    # Resize image to 512x512
    transform = transforms.Compose([
        transforms.Resize(512),
        transforms.CenterCrop(512),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Preprocess image
    image = transform(image).unsqueeze(0)

    # Move image to GPU if available
    return image.to(device)


# Define feature extraction function
def get_features(image, model):
    # Define layers to extract content and style features from
    content_layers = ['conv_4']
    style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

    # Dictionary to store content and style features
    features = {'content': {}, 'style': {}}

    # Loop through layers in model
    for name, layer in model._modules.items():
        image = layer(image)

        # If layer is in content_layers, store its features
        if name in content_layers:
            features['content'][name] = image

        # If layer is in style_layers, store its features
        if name in style_layers:
            features['style'][name] = gram_matrix(image)

    return features


# Define Gram matrix function
def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    features = tensor.view(b * c, h * w)
    gram = torch.mm(features, features.t())
    return gram.div(b * c * h * w)


# Define style transfer function
def style_transfer(content_image, style_image, num_iterations=1000, content_weight=1, style_weight=100000):
    # Preprocess content and style images
    content = preprocess(content_image)
    style = preprocess(style_image)

    # Extract content and style features
    content_features = get_features(content, model)
    style_features = get_features(style, model)


In [ ]:
Task 6

In [ ]:
from manim import *
import random


class BouncingBall(Scene):
    def construct(self):
        # Set up the ball
        ball = Circle(radius=0.5, fill_opacity=1, fill_color=BLUE).shift(UP * 3)

        # Set up the floor
        floor = Line(start=LEFT * 5, end=RIGHT * 5).shift(DOWN * 2.5)

        # Set up the initial velocity and acceleration of the ball
        velocity = 0
        acceleration = -9.8

        # Set up the time step
        dt = 0.1

        # Animate the ball
        while ball.get_bottom()[1] >= floor.get_top()[1]:
            # Update the position of the ball
            ball.shift(velocity * dt * UP + 0.5 * acceleration * dt ** 2 * DOWN)
            velocity += acceleration * dt

            # Randomly change the color of the ball
            ball.set_color(random.choice([RED, BLUE, GREEN, YELLOW]))

            self.add(ball, floor)
            self.wait(dt)

        self.play(FadeOut(ball))


In [ ]:
Task 7

In [ ]:
import tensorflow as tf
import numpy as np
import PIL.Image

# Load pre-trained InceptionV3 model
model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
model.trainable = False


# Define image preprocessing function
def preprocess(image):
    # Resize image to 299x299
    image = tf.image.resize(image, (299, 299))

    # Normalize image with ImageNet mean and standard deviation values
    image = tf.keras.applications.inception_v3.preprocess_input(image)

    return image


# Define layer loss function
def calculate_loss(layer_outputs):
    losses = []
    for output in layer_outputs:
        loss = tf.math.reduce_mean(tf.square(output))
        losses.append(loss)
    return tf.reduce_sum(losses)


# Define deep dream function
def deep_dream(image, octaves=6, octave_scale=1.3, iterations=10, step_size=0.01):
    # Convert image to a TensorFlow tensor
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = tf.keras.preprocessing.image.smart_resize(image, (224, 224))
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    image = tf.convert_to_tensor(image)

    # Define list of layers to use for optimization
    names = ['mixed3', 'mixed5', 'mixed7']
    layers = [model.get_layer(name).output for name in names]

    # Build the feature extraction model
    dream_model = tf.keras.Model(inputs=model.input, outputs=layers)

    # Define the octave scales
    octave_sizes = [tf.shape(image)[:2]]
    for i in range(1, octaves):
        size = tf.cast(tf.floor(octave_sizes[-1] / octave_scale), tf.int32)
        octave_sizes.append(size)

    # Generate the deep dream image
    for octave_size in octave_sizes[::-1]:
        # Resize the image to the current octave size
        image = tf.image.resize(image, octave_size)

        for i in range(iterations):
            # Optimize the image to maximize layer outputs
            with tf.GradientTape() as tape:
                tape.watch(image)
                outputs = dream_model(preprocess(image))
                loss = calculate_loss(outputs)

            gradients = tape.gradient(loss, image)
            gradients /= tf.math.reduce_std(gradients) + 1e-8
            image = image + gradients * step_size
            image = tf.clip_by


preprocess('IMG_7945.jpg')
